
### Глава 1 Элементарная теория вероятностей

**Кандидат должен знать:**
- классическое и геометрическое определение вероятности
- формуа полной вероятности и формула Байеса
- независимость/зависимость событий 
- Испытания Бернулли
- Теорема Пуассона
- теорема Муавра-Лапласа
- мат. ожидание в дискретном случае: $Ex = \sum p_i x_i$


**Материалы:**
- [Райгородский курс ТВ лекция 2 , 3](https://www.youtube.com/watch?v=n7jx2MfIeX4&list=PLthfp5exSWEr8tRK-Yf-i9aXgcFJ-O16d&index=2)
- [курс ТВ  на степике часть 1 и немного часть 2,3](https://stepik.org/course/3089)

## Задача с кодингом


**Disease Detection Dilemma**

A new disease has been discovered and there's a blood test that can detect it. However, there's a shortage of these tests. To maximize the number of people tested, a group testing strategy has been proposed: 

Take a group of $k$ people and test a mixture of their blood. If the test is negative, we've effectively tested $k$ people with just one test. If the test is positive, we then test each person in the group individually.

Your task is to determine the optimal size of the group $k$ that minimizes the expected number of tests.

**Tasks:**

1. Implement a function that calculates $p$ - the probability of a random person having the disease, given a binary array representing the disease status of a population.

2. Formulate mathematically the problem of finding the optimal group size $k$ given the disease probability $p$.

3. Write a function that calculates the optimal group size $k$ given the disease probability $p$.

4. Write a function that calculates the expected number of tests given the disease probability $p$, the total population size $n$, and the group size $k$.

**Bonus Tasks:**

5. The group testing strategy can be improved, especially for small group sizes. Think about how this could be done and modify tasks 3 and 4 to implement this improved strategy.

6. Write a function that, given the total population size $n$, the number of sick individuals $m$, and the group size $k$, calculates:
   - The expected number of tests.
   - The maximum number of tests (worst-case scenario).
   - The minimum number of tests (best-case scenario).




In [69]:
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm

## Task 1: Calculate Disease Probability

In [52]:
def calculate_disease_probability(population):
    # prob = population.count(1) / len(population)
    return np.mean(population)

## Task 2: Formulate the Problem

* **N** - total population size
* **k** - group size
* **p** - probability of a random person having the disease
* **a** = (1 - p) - probability of a random person not having the disease

We need $N_1 = 1$ test if no person in the group has the disease; the probability of this: $P_1 = (1 - p)^k = a^k$
We need $N_2 = k + 1$ tests if any person in the group has the disease; the probability: $P_2 = 1 - P_1 = 1 - a^k$

The expected number of tests in the group of \( k \) people: $E_k = N_1 \cdot P_1 + N_2 \cdot P_2$
$E_k = 1 \cdot a^k + (k + 1) \cdot (1 - a^k)$
$E_k = a^k + k + 1 - k \cdot a^k - a^k$
$E_k = 1 + k - k \cdot a^k$

To find expected tests num for all population N we need to multiply $E_k$ by number of groups $N_g = \frac{N}{k}$
$E = N_g \cdot E_k = \frac{N}{k} \cdot (1 + k - k \cdot a^k) = N \cdot (\frac{1}{k} + 1 - a^k) = N \cdot (\frac{1}{k} + 1 - (1-p)^k)$ 


## Task 4: Calculate Expected Number of Tests

In [ ]:
def calc_avg_expected_tests(p, n, k):
    if k == 1:
        return n
    return n*(1/k + 1 - (1-p)**k)

In [127]:
# tests
def generate_bin_array(size, p):
    return np.random.choice([0, 1], size=size, p=[1-p, p])

def count_tests(population, k):
    if k == 1:
        return len(population)
    tests = 0
    for i in range(0, len(population), k):
        group = population[i:i+k]
        if 1 in group:
            tests += 1 + k
        else:
            tests += 1
    return tests

def test():
    n_avg_tests = 2000
    for p_val in tqdm(np.linspace(0.04, 0.2, 7)):
        for k_val in range(5, 10):
            for _ in range(5):
                n_val = k_val * np.random.choice([1, 2, 4, 10, 15, 100, 1000])
                
                expected_tests = calc_avg_expected_tests(p_val, n_val, k_val)
                actual_avg_tests = 0
                for _ in range(n_avg_tests):
                    population = generate_bin_array(n_val, p_val)
                    actual_avg_tests += count_tests(population, k_val)
                try:
                    assert abs(expected_tests - actual_avg_tests/n_avg_tests) <= 2
                except AssertionError:
                    print(p_val, n_val, k_val, expected_tests, actual_avg_tests/n_avg_tests)


In [138]:
# 0.05263157894736842 30 5 13.10631117738022 10.9

calc_avg_expected_tests(0.05263157894736842, 10000, 5)
# 13.11
# N_t = 5000
# 
# actual_avg_tests = 0
# for _ in range(N_t):
#     population = generate_bin_array(30, 0.05263157894736842)
#     actual_avg_tests += count_tests(population, 5)
# 
# actual_avg_tests/N_t
# count_tests([1, 0, 0, 0, 0, 1], 2)

4368.770392460074

In [142]:
find_val(0.05263157894736842, 10000)

5

In [129]:
test()

  0%|          | 0/7 [00:00<?, ?it/s]

0.4 15000 5 16833.600000000002 16831.08


  0%|          | 0/7 [02:38<?, ?it/s]


KeyboardInterrupt: 

In [141]:
def find_val(p, n):
    prev = float("inf")
    for k in range(2, n+1):
        cur = calc_avg_expected_tests(p, n, k)
        if cur > prev:
            return k-1
        prev = cur
    return min(n, prev)

In [24]:
def get_y(p, n, k):
    y_values = n*(1-(1-p)**k + 1/k)
    y_values[k == 1] = n
    return y_values

N = 100
x = np.linspace(0, 1, 50)  # (p)
y = np.linspace(1, 100, 50)  # (k)
x, y = np.meshgrid(x, y)  # Create a meshgrid for x and z

z = get_y(x, N, y)

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title="avg_expected_tests",
                  scene=dict(
                      xaxis_title='p',
                      yaxis_title='people in group',
                      zaxis_title='N Tests'
                  ))
fig.show()

In [25]:
find_val(0.9, 4)

5.96 2
5.3293333333333335 3
4.9996 4


4

In [12]:
calc_avg_expected_tests(p=0.3, n=4, k=3)

3.961333333333333

[0 1 0 1 0 0 1 0 0 0]


## Альтернатива

### Circle Mystery: Identifying the Point Generation Algorithm

**Problem Statement:**

Peter has two algorithms that generate random points within a unit circle (centered at (0,0) with a radius of 1).

**Algorithm 1:**
This algorithm generates a point within the bounding square (i.e., $(x,y)$ where $x,y \sim U[-1, 1]$). If the point falls outside the circle, it re-generates point untill it falls inside the circle.

**Algorithm 2:**
This algorithm generates the radius and angle of the point (i.e., $ r \sim U[0,1]$ and $\alpha \sim [0, 2 \pi] $). It then calculates the coordinates of the point using $x = r \cos\alpha, y = r\sin\alpha $.

**Tasks:**

1. **Point Generation:** Implement two functions that generate $n$ points using the algorithms described above.

2. **Algorithm Prediction:** Implement a function that takes $n$ points as input and predicts which algorithm (1 or 2) was used to generate them.

3. **Prediction Quality:** Evaluate the quality of your algorithm prediction function:
   - Create a test dataset by generating $k$ sets of points using both algorithms (with $n=1000$ points in each set).
   - Use your algorithm prediction function to predict the algorithm used for each set of points in your test dataset.
   - Calculate the accuracy of your predictions.